<a href="https://colab.research.google.com/github/Sina76Sol/Conditional-Linear-Algebra-NN-/blob/main/CL_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
from keras import layers
from tensorflow import keras
import numpy as np

In [11]:
# Get model
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(128, activation="relu", name="dense_1")(inputs)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(10, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()


# define low rank approximation function
def low_rank_approx(weights, rank):
    u, s, vt = np.linalg.svd(weights, full_matrices=False)
    s = np.diag(s)
    approximation = u[:, :rank] @ s[0:rank, :rank] @ vt[:rank, :]
    return approximation


# define apply_approximation function
def apply_low_rank(model_weights, rank):
    for w in model_weights:
        if w.shape.rank < 2:
            pass
        else:
            approx = low_rank_approx(w.numpy(), rank)
            w.assign(tf.Variable(approx))


# Creating a function to do the training and validation and return the accuracy values and error:
def general(epochs, train, val, r):
    acc = []
    for epoch in range(epochs):
        epoch_acc = []
        print("\nStart of epoch %d" % (epoch,))

        # Iterate over the batches of the dataset.
        for step, (x_batch_train, y_batch_train) in enumerate(train):
            with tf.GradientTape() as tape:
                logits = model(x_batch_train, training=True)
                loss_value = loss_fn(y_batch_train, logits)
            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))
            apply_low_rank(model.trainable_weights, r)
            # Update training metric.
            train_acc_metric.update_state(y_batch_train, logits)

            # Log every 200 batches.
            # if step % 200 == 0:
            #     print(
            #         "Training loss (for one batch) at step %d: %.4f"
            #         % (step, float(loss_value))
            #     )
            #     print("Seen so far: %d samples" % ((step + 1) * batch_size))

        # Display metrics at the end of each epoch.
        train_acc = train_acc_metric.result()

        # Reset training metrics at the end of each epoch
        train_acc_metric.reset_states()
        epoch_acc.append(float(train_acc))

        # Run a validation loop at the end of each epoch.
        for x_batch_val, y_batch_val in val:
            val_logits = model(x_batch_val, training=False)
            # Update val metrics
            val_acc_metric.update_state(y_batch_val, val_logits)
        val_acc = val_acc_metric.result()
        epoch_acc.append(float(val_acc))
        val_acc_metric.reset_states()
        acc.append(epoch_acc)
    return acc

In [12]:
# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

11490434/11490434 [==============================] - 0s 0us/step


In [13]:
import matplotlib.pyplot as plt

In [ ]:
rank = [1, 5, 10, 15, 20, 128]
rate =[]
for r in rank:
  rate = general(5, train_dataset, val_dataset, r)
  rate = np.array(rate)
  plt.plot(rate[:, 0], label='training accuracy for r = ' + str(r))
  plt.plot(rate[:, 1], label='validation accuracy for r = ' + str(r))
  plt.legend()
plt.rcParams['figure.figsize'] = [10, 8]
plt.show()


Start of epoch 0

Start of epoch 1

Start of epoch 2


In [ ]:
train_rate = general(5, train_dataset, val_dataset, 5)

In [ ]:
train_rate = np.array(train_rate)
a=0
plt.plot(train_rate[:, 0], label='training accuracy')
plt.plot(train_rate[:, 1], label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
x = [1,2,3]
plt.figure(1)
plt.rcParams['figure.figsize'] = [10, 5]
plt.plot(x)
plt.show()